### Ejercicio 20: Alquilatos

In [56]:
import pandas as pd
import numpy as np
from pyomo.environ import *
model = ConcreteModel()

In [57]:
df = pd.DataFrame({'A1': [25,60,75], 'A2': [25,50,85]}, index = ['C1', 'C2', 'C3'])

model.p = Set(initialize = df.columns)
model.c = Set(initialize = df.index)

model.prodmax = Param(model.p, initialize = {'A1': 1.6 , 'A2': 0.8})
model.demmin = Param(model.c, initialize = {'C1': 0.9 , 'C2': 0.7, 'C3': 0.3})
model.costeprod = Param(model.p, initialize = {'A1': 30 , 'A2': 35})

df

,A1,A2
C1,25,25
C2,60,50
C3,75,85


In [58]:
model.F = Var(model.p, model.c, domain= NonNegativeReals)

In [59]:
model.cost = Objective(expr = sum(model.F[i,j]*df[i][j] for i in model.p for j in model.c)+sum(model.costeprod[i]*sum(model.F[i,j] for j in model.c) for i in model.p))

In [60]:
model.pmax = ConstraintList()
for i in model.p:
    model.pmax.add(
        sum(model.F[i,j] for j in model.c) <= model.prodmax[i]
    )
model.dmin = ConstraintList()
for j in model.c:
    model.dmin.add(
        sum(model.F[i,j] for i in model.p) >= model.demmin[j]
    )

In [61]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write() 

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 140.5
  Upper bound: 140.5
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 13
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05386686325073242
# ----------------------------------------------------------
#   Solution Information
# ------------------------------------